# Initial Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np

## MAKE A CNN


# Load in Data 

In [ ]:

malta_loc_18 = '../data/Malta-Budget-2018-dataset-v1.csv'
malta_loc_19 = '../data/Malta-Budget-2019-dataset-v1.csv'
malta_loc_20 = '../data/Malta-Budget-2020-dataset-v1.csv'

malta_data_18 = pd.read_csv(malta_loc_18)
malta_data_19 = pd.read_csv(malta_loc_19)
malta_data_20 = pd.read_csv(malta_loc_20)

print(malta_data_18.info())
print(malta_data_19.info())
print(malta_data_20.info())

# Merge data files into single DataFrame

In [ ]:
malta_data_19 = malta_data_19.rename(columns={'Off-topic ':'Off-topic'})
combined_data = pd.concat([malta_data_18, malta_data_19, malta_data_20], ignore_index=True)
combined_data.info()

# Clean up of data columns 

In [ ]:
clean_data = combined_data.dropna(subset=['Online Post Text'])
clean_data = clean_data.drop(['Twitter ID', 'Related Online Post ID', 'Source ID','Off-topic'], axis=1)
clean_data = clean_data[clean_data['Language'] == 0] # get all data that is in english 
clean_data = clean_data.drop(['Language'], axis=1)
clean_data = clean_data.rename(columns={'Online Post ID':'ID','Online Post Text':'Text'})
clean_data.head()

In [ ]:
clean_data.info()

# Emotion data Exploration

In [ ]:
clean_data["Emotion"].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

value_counts = clean_data['Emotion'].value_counts()

# Plotting the frequency of string values in the 'Category' column
plt.figure(figsize=(10, 6))
sns.barplot(x=value_counts.index, y=value_counts.values, palette='viridis', hue=value_counts.index, legend=False)
plt.xlabel('Emotion')
plt.ylabel('Frequency')
plt.title('Frequency of Emotional Categories In English Comments about The Maltese Budget 2018-2020')
plt.show()

# Pre Processing of Text

In [ ]:
import string
import re
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions

def remove_special_characters(text):
    pattern = re.compile(r'[^a-zA-Z\s]')
    return pattern.sub('', text)

# Remove URLs and HTML tags
clean_data['Text'] = clean_data['Text'].str.replace(r'http\S+|www\S+|https\S+', '', regex=True)
clean_data['Text'] = clean_data['Text'].str.replace(r'<.*?>', '', regex=True)

# Expand contractions
clean_data['Text'] = clean_data['Text'].apply(lambda x: contractions.fix(x))

# Convert to lowercase
clean_data['Text'] = clean_data['Text'].str.lower()

# Remove punctuation
clean_data['Text'] = clean_data['Text'].str.replace(f"[{string.punctuation}]", " ", regex=True)

# Remove numbers
clean_data['Text'] = clean_data['Text'].str.replace(r'\d+', '', regex=True)

# Remove special characters
clean_data['Text'] = clean_data['Text'].apply(remove_special_characters)

# Remove stop words
stop_words = set(stopwords.words('english'))
clean_data['Text'] = clean_data['Text'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

# Remove extra whitespace
clean_data['Text'] = clean_data['Text'].str.strip()
clean_data['Text'] = clean_data['Text'].str.replace(r'\s+', ' ', regex=True)

# Lemmatize
lemmatizer = WordNetLemmatizer()
clean_data['Text'] = clean_data['Text'].apply(lambda x: ' '.join(lemmatizer.lemmatize(word) for word in x.split()))

# Tokenize
# clean_data['tokens'] = clean_data['Text'].apply(word_tokenize)
 
clean_data.head()

In [ ]:
stripped_data = clean_data.drop(columns=['Subjectivity','Sentiment Polarity', 'Sarcasm','Irony','Negation'])
stripped_data.head()

# Multinomial Naive Bayes

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

train_df, test_df = train_test_split(stripped_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_df['Text'])
X_test = vectorizer.transform(test_df['Text'])

# Labels
y_train = train_df['Emotion']
y_test = test_df['Emotion']

# Training Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train, y_train)

# Predicting on test data
y_pred = model.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.28699551569506726
Classification Report:
               precision    recall  f1-score   support

       anger       0.19      0.31      0.24        45
anticipation       0.43      0.45      0.44        40
     disgust       0.32      0.51      0.39        51
        fear       0.50      0.11      0.18         9
         joy       0.00      0.00      0.00        19
     sadness       0.29      0.18      0.22        28
    surprise       0.00      0.00      0.00        18
       trust       0.00      0.00      0.00        13

    accuracy                           0.29       223
   macro avg       0.22      0.20      0.18       223
weighted avg       0.25      0.29      0.25       223



# CNN with Tokenisation

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D, MaxPooling2D, Conv2D

# Tokenization and Padding
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(stripped_data['Text'])
sequences = tokenizer.texts_to_sequences(stripped_data['Text'])
x_data = pad_sequences(sequences, maxlen=max_len)
y_data = pd.get_dummies(stripped_data['Emotion']).values  # Convert categorical labels to one-hot encoded vectors

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)
print('Shape of training data: ')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data: ')
print(X_test.shape)
print(y_test.shape)

# CNN Model Architecture
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim))  # Specify input_dim instead of input_length
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1], activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with One Hot Encoding 

In [ ]:
# Tokenization and Padding using one_hot
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)
encoded_docs = [one_hot(d, max_features) for d in stripped_data['Text']]
x_data = pad_sequences(encoded_docs, maxlen=max_len)
y_data = pd.get_dummies(stripped_data['Emotion']).values  # Convert categorical labels to one-hot encoded vectors

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)
print('Shape of training data: ')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data: ')
print(X_test.shape)
print(y_test.shape)

# CNN Model Architecture
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim))  # Specify input_dim instead of input_length
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1], activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with label encoding

# CNN with Oridinal encoding

# CNN with VAE endcoding

# CNN with BagOfWords

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Bag-of-Words Vectorization
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)

count_vectorizer = CountVectorizer(max_features=max_features)
x_data = count_vectorizer.fit_transform(stripped_data['Text']).toarray()

# Convert labels to one-hot encoded vectors
y_data = pd.get_dummies(stripped_data['Emotion']).values

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)

print('Shape of training data:')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data:')
print(X_test.shape)
print(y_test.shape)

# CNN Model Architecture
embedding_dim = 100
model = Sequential()
model.add(Dense(embedding_dim, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Reshape((embedding_dim, 1)))  # Reshape to match Conv1D input shape
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1], activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import Reshape

# TF-IDF Vectorization
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)

tfidf = TfidfVectorizer(max_features=max_features)
x_data = tfidf.fit_transform(stripped_data['Text']).toarray()

# Convert labels to one-hot encoded vectors
y_data = pd.get_dummies(stripped_data['Emotion']).values

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)

print('Shape of training data:')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data:')
print(X_test.shape)
print(y_test.shape)

# CNN Model Architecture
embedding_dim = 100
model = Sequential()
model.add(Dense(embedding_dim, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Reshape((embedding_dim, 1)))  # Reshape to match Conv1D input shape
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1], activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with N-Grams

In [ ]:
# Tokenization and Padding using Tokenizer with n-grams
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)
n_gram_range = 2  # Unigram and bigram

# Function to add n-grams
def create_ngram_set(input_list, ngram_value=2):
    return set(zip(*[input_list[i:] for i in range(ngram_value)]))

def add_ngram(sequences, token_index, ngram_range=2):
    new_sequences = []
    for input_list in sequences:
        new_list = input_list[:]
        for ngram_value in range(2, ngram_range + 1):
            for ngram in create_ngram_set(input_list, ngram_value):
                if ngram in token_index:
                    new_list.append(token_index[ngram])
                else:
                    # Add a new token to the index
                    token_index[ngram] = len(token_index) + 1
                    new_list.append(token_index[ngram])
        new_sequences.append(new_list)
    return new_sequences

# Tokenizer initialization and fitting
tokenizer = Tokenizer(num_words=max_features, oov_token="<OOV>")
tokenizer.fit_on_texts(stripped_data['Text'])

# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(stripped_data['Text'])

# Add n-gram features
token_index = tokenizer.word_index.copy()  # Copy the word index
sequences_ngram = add_ngram(sequences, token_index, n_gram_range)

# Pad sequences
x_data = pad_sequences(sequences_ngram, maxlen=max_len)

# Convert labels to one-hot encoded vectors
y_data = pd.get_dummies(stripped_data['Emotion']).values

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)

print('Shape of training data:')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data:')
print(X_test.shape)
print(y_test.shape)

# CNN Model Architecture
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=len(token_index) + 1, output_dim=embedding_dim))  # Adjust input_dim
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1], activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN Mark II

In [ ]:
# Tokenization and Padding using one_hot
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)
encoded_docs = [one_hot(d, max_features) for d in stripped_data['Text']]
x_data = pad_sequences(encoded_docs, maxlen=max_len)
y_data = pd.get_dummies(stripped_data['Emotion']).values  # Convert categorical labels to one-hot encoded vectors

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)
print('Shape of training data: ')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data: ')
print(X_test.shape)
print(y_test.shape)

# CNN Model Architecture
embedding_dim = 100
model = Sequential()
model.add(Input(shape=(max_len,), dtype='int32'))
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim))  # Specify input_dim instead of input_length
model.add(Conv1D(100, 2, activation='relu'))
# model.add(Dropout(0.2))
model.add(Conv1D(100, 3, activation='relu'))
# model.add(Dropout(0.2))
model.add(Conv1D(100, 4, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(MaxPooling1D(pool_size=2))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.1))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1], activation='softmax')) 

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with SMOTE (for undistributed data)

In [ ]:
from imblearn.over_sampling import SMOTE

# Tokenization and Padding using one_hot
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)
encoded_docs = [one_hot(d, max_features) for d in stripped_data['Text']]
x_data = pad_sequences(encoded_docs, maxlen=max_len)
y_data = pd.get_dummies(stripped_data['Emotion']).values  # Convert categorical labels to one-hot encoded vectors

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)
print('Shape of training data: ')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data: ')
print(X_test.shape)
print(y_test.shape)

smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# CNN Model Architecture
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim))  # Specify input_dim instead of input_length
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1], activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20#30
epochs = 20
model.fit(X_train_resampled, y_train_resampled, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with Class Weights (for undistributed data) 

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Tokenization and Padding using one_hot
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)
encoded_docs = [one_hot(d, max_features) for d in stripped_data['Text']]
x_data = pad_sequences(encoded_docs, maxlen=max_len)
y_data = pd.get_dummies(stripped_data['Emotion']).values  # Convert categorical labels to one-hot encoded vectors

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)
print('Shape of training data: ')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data: ')
print(X_test.shape)
print(y_test.shape)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(stripped_data['Emotion']), y=stripped_data['Emotion'])
class_weights = dict(enumerate(class_weights))

# CNN Model Architecture
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim))  # Specify input_dim instead of input_length
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1], activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), class_weight=class_weights)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with ADASYN

In [ ]:
from imblearn.over_sampling import ADASYN

# Tokenization and Padding using one_hot
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)
encoded_docs = [one_hot(d, max_features) for d in stripped_data['Text']]
x_data = pad_sequences(encoded_docs, maxlen=max_len)
y_data = pd.get_dummies(stripped_data['Emotion']).values  # Convert categorical labels to one-hot encoded vectors

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)
print('Shape of training data: ')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data: ')
print(X_test.shape)
print(y_test.shape)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(stripped_data['Emotion']), y=stripped_data['Emotion'])
class_weights = dict(enumerate(class_weights))

adasyn = ADASYN(sampling_strategy='minority')
X_res, y_res = adasyn.fit_resample(X_train, y_train)

# CNN Model Architecture
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim))  # Specify input_dim instead of input_length
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1], activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit(X_res, y_res, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), class_weight=class_weights)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with NearMiss

In [ ]:
from imblearn.under_sampling import NearMiss

# Tokenization and Padding using one_hot
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)
encoded_docs = [one_hot(d, max_features) for d in stripped_data['Text']]
x_data = pad_sequences(encoded_docs, maxlen=max_len)
y_data = pd.get_dummies(stripped_data['Emotion']).values  # Convert categorical labels to one-hot encoded vectors

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)
print('Shape of training data: ')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data: ')
print(X_test.shape)
print(y_test.shape)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(stripped_data['Emotion']), y=stripped_data['Emotion'])
class_weights = dict(enumerate(class_weights))

nearmiss = NearMiss(version=1)
X_res, y_res = nearmiss.fit_resample(X_train, y_train)

# CNN Model Architecture
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim))  # Specify input_dim instead of input_length
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1], activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit(X_res, y_res, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), class_weight=class_weights)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with Borderline_SMOTE

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE

# Tokenization and Padding using one_hot
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)
encoded_docs = [one_hot(d, max_features) for d in stripped_data['Text']]
x_data = pad_sequences(encoded_docs, maxlen=max_len)
y_data = pd.get_dummies(stripped_data['Emotion']).values  # Convert categorical labels to one-hot encoded vectors

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)
print('Shape of training data: ')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data: ')
print(X_test.shape)
print(y_test.shape)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(stripped_data['Emotion']), y=stripped_data['Emotion'])
class_weights = dict(enumerate(class_weights))

borderline_smote = BorderlineSMOTE()
X_res, y_res = borderline_smote.fit_resample(X_train, y_train)

# CNN Model Architecture
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim))  # Specify input_dim instead of input_length
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1], activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit(X_res, y_res, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), class_weight=class_weights)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with SMOTE + Tomek Links (SMOTETomek)

In [ ]:
from imblearn.combine import SMOTETomek

# Tokenization and Padding using one_hot
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)
encoded_docs = [one_hot(d, max_features) for d in stripped_data['Text']]
x_data = pad_sequences(encoded_docs, maxlen=max_len)
y_data = pd.get_dummies(stripped_data['Emotion']).values  # Convert categorical labels to one-hot encoded vectors

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)
print('Shape of training data: ')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data: ')
print(X_test.shape)
print(y_test.shape)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(stripped_data['Emotion']), y=stripped_data['Emotion'])
class_weights = dict(enumerate(class_weights))

smote_tomek = SMOTETomek()
X_res, y_res = smote_tomek.fit_resample(X_train, y_train)

# CNN Model Architecture
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim))  # Specify input_dim instead of input_length
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1], activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit(X_res, y_res, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), class_weight=class_weights)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with Word2Vec embedding

In [ ]:
from gensim.models import KeyedVectors

# Path to the pre-trained Word2Vec file
word2vec_path = '../data/GoogleNews-vectors-negative300.bin'

# Load Word2Vec model
word2vec = KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [ ]:
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)

# Assuming 'stripped_data' contains your text data
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(stripped_data['Text'])

sequences = tokenizer.texts_to_sequences(stripped_data['Text'])
x_data = pad_sequences(sequences, maxlen=max_len)

# Convert categorical labels to one-hot encoded vectors
y_data = pd.get_dummies(stripped_data['Emotion']).values

word_index = tokenizer.word_index
num_words = min(max_features, len(word_index) + 1)
embedding_dim = 300  # Assuming your Word2Vec model has embeddings of size 300

embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i >= max_features:
        continue
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]
    # If word not found in embedding index, leave it as zero-vector or initialize randomly
    
# Optionally, you can normalize the embedding matrix
# embedding_matrix = embedding_matrix / np.linalg.norm(embedding_matrix, axis=-1, keepdims=True)

model = Sequential()
model.add(Embedding(num_words, embedding_dim, weights=[embedding_matrix], trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)

batch_size = 20
epochs = 20

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN GloVe embedding


In [ ]:
# Load GloVe embeddings
embedding_dim = 100
# gloVe_loc = "../data/glove.840B.300d.txt"  # Path to GloVe file
gloVe_loc = "../data/glove.6B.100d.txt"
embeddings_index = {}

# Determine embedding dimension automatically
with open(gloVe_loc, 'r', encoding='utf-8') as f:
    first_line = f.readline()
    embedding_dim = len(first_line.split()) - 1  # The number of dimensions is one less than the number of columns

# Reload the GloVe file to build the embeddings_index
with open(gloVe_loc, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        except ValueError:
            print(f"Skipping line: {line}")
            
# Tokenization and Padding using one_hot
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)
encoded_docs = [one_hot(d, max_features) for d in stripped_data['Text']]
x_data = pad_sequences(encoded_docs, maxlen=max_len)
y_data = pd.get_dummies(stripped_data['Emotion']).values  # Convert categorical labels to one-hot encoded vectors

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)
print('Shape of training data: ')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data: ')
print(X_test.shape)
print(y_test.shape)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(stripped_data['Emotion']), y=stripped_data['Emotion'])
class_weights = dict(enumerate(class_weights))

# Create embedding matrix
embedding_matrix = np.zeros((max_features, embedding_dim))
word_index = {word: i for i, word in enumerate(embeddings_index.keys(), 1)}  # Build word index
for word, i in word_index.items():
    if i < max_features:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Build CNN Model with GloVe embeddings
model = Sequential()
model.add(Input(shape=(max_len,), dtype='int32'))
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim, weights=[embedding_matrix], trainable=False))  # Pre-trained GloVe embeddings
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_data.shape[1], activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 32
epochs = 20
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), class_weight=class_weights)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with FastText

# CNN with LSTM

In [ ]:
from keras.layers import LSTM
# Tokenization and Padding using one_hot
max_features = 10000  # Max number of words in tokenizer
max_len = 100  # Max length of each sequence (pad/truncate to this length)

tokenizer = Tokenizer(num_words=max_features, oov_token="<OOV>")
tokenizer.fit_on_texts(stripped_data['Text'])

# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(stripped_data['Text'])

# Pad sequences
x_data = pad_sequences(sequences, maxlen=max_len)

# Convert labels to one-hot encoded vectors
y_data = pd.get_dummies(stripped_data['Emotion']).values

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)

print('Shape of training data:')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data:')
print(X_test.shape)
print(y_test.shape)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(stripped_data['Emotion']), y=stripped_data['Emotion'])
class_weights = dict(enumerate(class_weights))

# LSTM Model Architecture
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_data.shape[1], activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), class_weight=class_weights)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with Contextualised Embedding

# CNN with BERT

In [ ]:
from transformers import BertTokenizer, TFBertModel
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode sequences in the dataset
def encode_texts(texts, tokenizer, max_len):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_token_type_ids=False,
            truncation=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return np.array(input_ids), np.array(attention_masks)

max_len = 100  # Max length of each sequence
input_ids, attention_masks = encode_texts(stripped_data['Text'].tolist(), tokenizer, max_len)

# Convert labels to one-hot encoded vectors
y_data = pd.get_dummies(stripped_data['Emotion']).values

# Splitting data into training and testing sets
X_train_ids, X_test_ids, y_train, y_test = train_test_split(input_ids, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)
X_train_masks, X_test_masks, _, _ = train_test_split(attention_masks, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)

print('Shape of training data:')
print(X_train_ids.shape)
print(y_train.shape)
print('Shape of test data:')
print(X_test_ids.shape)
print(y_test.shape)

# Build the model
input_ids = Input(shape=(max_len,), dtype='int32', name='input_ids')
input_masks = Input(shape=(max_len,), dtype='int32', name='attention_masks')

bert_output = bert_model(input_ids, attention_mask=input_masks)[0]
x = LSTM(128, return_sequences=True)(bert_output)
x = Dropout(0.2)(x)
x = LSTM(128)(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
output = Dense(y_data.shape[1], activation='softmax')(x)  # Output layer with softmax activation for multi-class classification

model = Model(inputs=[input_ids, input_masks], outputs=output)

# Compile the model
optimizer = Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 16
epochs = 3
model.fit([X_train_ids, X_train_masks], y_train, batch_size=batch_size, epochs=epochs, validation_data=([X_test_ids, X_test_masks], y_test))

# Evaluate the model
loss, accuracy = model.evaluate([X_test_ids, X_test_masks], y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with ELMo

In [ ]:
from allennlp.modules.elmo import Elmo, batch_to_ids
# Load ELMo model
options_file = 'https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway_options.json'
weight_file = 'https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway_weights.hdf5'
elmo = Elmo(options_file, weight_file, 2, dropout=0)

# Tokenize text and convert to character ids
def tokenize_texts(texts):
    sentences = [text.split() for text in texts]
    character_ids = batch_to_ids(sentences)
    return character_ids

# Convert texts to character ids for ELMo
character_ids = tokenize_texts(stripped_data['Text'].tolist())

# Extract ELMo embeddings
def get_elmo_embeddings(character_ids):
    embeddings = elmo(character_ids)['elmo_representations'][0]
    return embeddings.numpy()

elmo_embeddings = get_elmo_embeddings(character_ids)

# Average the ELMo embeddings across tokens
def average_elmo_embeddings(elmo_embeddings):
    return np.mean(elmo_embeddings, axis=1)

x_data = average_elmo_embeddings(elmo_embeddings)

# Convert labels to one-hot encoded vectors
y_data = pd.get_dummies(stripped_data['Emotion']).values

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)

print('Shape of training data:')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data:')
print(X_test.shape)
print(y_test.shape)

# Build the model
input_dim = X_train.shape[1]  # Number of features in the ELMo embeddings

input_layer = Input(shape=(input_dim,))
x = Dense(128, activation='relu')(input_layer)
x = Dropout(0.2)(x)
x = LSTM(128)(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
output = Dense(y_data.shape[1], activation='softmax')(x)  # Output layer with softmax activation for multi-class classification

model = Model(inputs=input_layer, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# CNN with RoBERTa

In [ ]:
from transformers import RobertaTokenizer, TFRobertaModel

# Load pre-trained RoBERTa model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

# Tokenize and encode sequences in the dataset
def encode_texts(texts, tokenizer, max_len):
    encodings = tokenizer(
        texts, 
        truncation=True, 
        padding=True, 
        max_length=max_len, 
        return_tensors='tf'
    )
    return encodings['input_ids'], encodings['attention_mask']

max_len = 100  # Max length of each sequence
input_ids, attention_masks = encode_texts(stripped_data['Text'].tolist(), tokenizer, max_len)

# Convert labels to one-hot encoded vectors
y_data = pd.get_dummies(stripped_data['Emotion']).values

# Splitting data into training and testing sets
X_train_ids, X_test_ids, y_train, y_test = train_test_split(input_ids, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)
X_train_masks, X_test_masks, _, _ = train_test_split(attention_masks, y_data, test_size=0.2, stratify=stripped_data['Emotion'], random_state=42)

print('Shape of training data:')
print(X_train_ids.shape)
print(y_train.shape)
print('Shape of test data:')
print(X_test_ids.shape)
print(y_test.shape)

# Build the model
input_ids_layer = Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
attention_mask_layer = Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')

roberta_output = roberta_model(input_ids_layer, attention_mask=attention_mask_layer)[0]
x = tf.reduce_mean(roberta_output, axis=1)  # Average pooling over the sequence
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = LSTM(128)(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
output = Dense(y_data.shape[1], activation='softmax')(x)  # Output layer with softmax activation for multi-class classification

model = Model(inputs=[input_ids_layer, attention_mask_layer], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
batch_size = 20
epochs = 20
model.fit([X_train_ids, X_train_masks], y_train, batch_size=batch_size, epochs=epochs, validation_data=([X_test_ids, X_test_masks], y_test))

# Evaluate the model
loss, accuracy = model.evaluate([X_test_ids, X_test_masks], y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)